In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/cleandata/dataset_clean-2.xlsx


In [2]:
# kütüphaneleri import edelim
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import re
import seaborn as sns

# pythonda uyarıları kapatır
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install rapidfuzz


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 46.2 MB/s eta 0:00:00


In [4]:
# ==================== Text Standardization Pipeline (TR -> ASCII Output) ====================
# !pip install rapidfuzz openpyxl

import re
import unicodedata
import numpy as np
import pandas as pd
from collections import Counter

try:
    from rapidfuzz import process, fuzz
except Exception:
    raise RuntimeError("Lütfen 'rapidfuzz' paketini kurun: pip install rapidfuzz")

# --------- Ayarlar ---------
DATA_PATH = "/kaggle/input/cleandata/dataset_clean-2.xlsx"  # gerekirse değiştirin
SHEET_NAME = 0  # 'Sheet1' olabilir

# Çoklu/tek değerli kolonlar
COLS_MULTI  = ["kronik_hastalik", "alerji", "tanilar", "uygulama_yerleri"]
COLS_SINGLE = []  # 'bolum' işlenmiyor

# Kanonik kümesine girecek adayların minimum frekansı
MIN_COUNT_DEFAULT = 5

# Fuzzy eşik: 0-100; ne kadar yüksekse o kadar katı
FUZZY_THRESHOLD = 88

# (Opsiyonel) manuel eş anlamlı/haritalar: soldaki -> sağdaki kanonik (tümü lower-case olmalı)
MANUAL_ALIASES = {
    # "alerji": {"penisilin": "penisilin alerjisi"},
    # "tanilar": {"herni": "disk hernisi"},
}

# Çıktı dosya adları
OUT_STD_ALL = "Clean_Data_Case_DT_2025_std.xlsx"    # orijinal + *_std sütunları
OUT_CLEAN   = "Clean_Data_Case_DT_2025_clean.xlsx"  # kolon sırası AYNI, içerik temiz

# Çıktıyı ASCII (İngilizce karakter) ister misin?
OUTPUT_ASCII = True

# --------- Yardımcılar ---------
TR_KEEP = "a-zA-Z0-9çğıöşüÇĞİÖŞÜıİ"
NON_WORDS = re.compile(fr"[^\s{TR_KEEP}]+")

def std_space(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def tr_lower(s):
    """Türkçe'ye duyarlı küçük harfe çevirme: İ->i, I->ı, sonra .lower()"""
    if pd.isna(s):
        return np.nan
    s = str(s)
    s = s.replace("İ", "i").replace("I", "ı")
    return s.lower()

def normalize_basic(s: str) -> str:
    """Boşluk, noktalama sadeleştirme, TR karakterlerini koruyarak basit normalize."""
    if pd.isna(s): 
        return ""
    s = str(s)
    s = tr_lower(s)
    s = s.replace("-", " ").replace("–", " ").replace("—", " ")
    s = NON_WORDS.sub(" ", s)     # TR harfleri hariç noktalama temizliği
    s = std_space(s)
    return s

def to_ascii(text: str) -> str:
    """Türkçe karakterleri ASCII'ye çevir + diakritik temizliği"""
    if text is None or (isinstance(text, float) and np.isnan(text)):
        return text
    t = str(text)
    # Önce genel diakritik temizliği (örn: â -> a)
    t_norm = unicodedata.normalize("NFD", t)
    t_no_diac = "".join(ch for ch in t_norm if unicodedata.category(ch) != "Mn")
    # TR özgü harfler (lower case bekliyoruz ama yine de tüm varyantları kapsayalım)
    tr_map = {
        "ç": "c", "Ç": "C",
        "ğ": "g", "Ğ": "G",
        "ı": "i", "I": "I",  # I olduğu gibi kalabilir; metin zaten lower'da i/ı'ya dönüyor
        "ö": "o", "Ö": "O",
        "ş": "s", "Ş": "S",
        "ü": "u", "Ü": "U",
    }
    return t_no_diac.translate(str.maketrans(tr_map))

def fold_key(s: str) -> str:
    """Fuzzy eşleştirme için diakritik/işaretlerden arındırılmış karşılaştırma anahtarı."""
    s = normalize_basic(s)
    s_norm = unicodedata.normalize("NFD", s)
    s_no_diac = "".join(ch for ch in s_norm if unicodedata.category(ch) != "Mn")
    s_no_diac = s_no_diac.replace("ı", "i")  # karşılaştırma anahtarında ı->i
    return std_space(s_no_diac)

def split_multi_cell(x: str) -> list:
    """Virgül/; / / | ile ayrılmış çoklu değerleri parçala."""
    if pd.isna(x):
        return []
    s = str(x)
    parts = re.split(r"[;,/|]", s)
    parts = [p.strip() for p in parts if p and p.strip()]
    return parts

def combine_tokens(tokens: list) -> str:
    """Tekilleştir, sırala ve ', ' ile birleştir."""
    toks = sorted(set([t for t in tokens if t]))
    return ", ".join(toks) if toks else np.nan

def build_canonical(series: pd.Series, is_multi: bool, min_count: int) -> list:
    """Seriden (normalize edilmiş) aday terimleri say ve kanonik küme oluştur."""
    cnt = Counter()
    if is_multi:
        for val in series.dropna().astype(str).values:
            for t in split_multi_cell(val):
                t = normalize_basic(t)
                if t:
                    cnt[t] += 1
    else:
        for val in series.dropna().astype(str).values:
            t = normalize_basic(val)
            if t:
                cnt[t] += 1
    canonical = [t for t, c in cnt.most_common() if c >= min_count]
    return canonical

def make_mapper(canonical_terms: list, manual_alias: dict | None, fuzzy_threshold: int):
    """Token -> kanonik eşleyici döndürür. Önce manuel, sonra fuzzy. Çıkış ASCII olabilir."""
    canon_keys = [fold_key(c) for c in canonical_terms]
    canon_lookup = dict(zip(canon_keys, canonical_terms))  # key -> canonical (lower-case)
    
    manual_alias = manual_alias or {}
    manual_norm = {normalize_basic(k): normalize_basic(v) for k, v in manual_alias.items()}

    def _finish(token: str) -> str:
        return to_ascii(token) if OUTPUT_ASCII else token

    def map_one(token: str) -> str:
        if not token:
            return ""
        t_norm = normalize_basic(token)
        if not t_norm:
            return ""
        # 1) Manuel alias
        if t_norm in manual_norm:
            return _finish(manual_norm[t_norm])
        # 2) Direkt kanonikte varsa
        if t_norm in canonical_terms:
            return _finish(t_norm)
        # 3) Fuzzy en yakın kanonik
        if canon_keys:
            key = fold_key(t_norm)
            match = process.extractOne(key, canon_keys, scorer=fuzz.token_sort_ratio)
            if match:
                best_key, score, _ = match
                if score >= fuzzy_threshold:
                    return _finish(canon_lookup[best_key])
        # 4) Eşik altı ise normalize edilmiş haliyle bırak
        return _finish(t_norm)

    return map_one

def standardize_column(series: pd.Series, is_multi: bool, min_count: int, manual_alias: dict | None) -> pd.Series:
    """Kolonu kanonize eder ve *_std serisi döndürür (çıktı ASCII olabilir)."""
    canonical_terms = build_canonical(series, is_multi=is_multi, min_count=min_count)
    mapper = make_mapper(canonical_terms, manual_alias, FUZZY_THRESHOLD)

    if is_multi:
        out = []
        for val in series.values:
            tokens = split_multi_cell(val) if pd.notna(val) else []
            mapped = [mapper(t) for t in tokens]
            out.append(combine_tokens(mapped))
        return pd.Series(out, index=series.index, dtype="object")
    else:
        return series.apply(lambda x: mapper(x) if pd.notna(x) else np.nan)

# --------- Ana akış ---------
def run_standardization(
    df: pd.DataFrame,
    cols_multi=COLS_MULTI,
    cols_single=COLS_SINGLE,
    min_count_default=MIN_COUNT_DEFAULT,
    manual_aliases=MANUAL_ALIASES
) -> pd.DataFrame:
    """Orijinal df + *_std kolonlarıyla birlikte döner (orijinal kolon sırası korunur)."""
    df = df.copy()

    # Çoklu kolonlar
    for col in cols_multi:
        if col in df.columns:
            aliases = manual_aliases.get(col.lower(), {})
            df[col + "_std"] = standardize_column(
                df[col],
                is_multi=True,
                min_count=min_count_default,
                manual_alias=aliases
            )

    # Tek kolonlar (şu an boş)
    for col in cols_single:
        if col in df.columns:
            aliases = manual_aliases.get(col.lower(), {})
            df[col + "_std"] = standardize_column(
                df[col],
                is_multi=False,
                min_count=min_count_default,
                manual_alias=aliases
            )

    return df

def make_clean_copy_keep_order(df_original: pd.DataFrame,
                               df_std: pd.DataFrame,
                               cols_multi=COLS_MULTI,
                               cols_single=COLS_SINGLE) -> pd.DataFrame:
    """
    Kolon SIRASINI KORUYARAK temiz kopya üretir:
      - Orijinal kolon adları ve POZİSYONLARI aynı kalır.
      - Hedef kolonların (multi+single) İÇERİĞİ, *_std karşılıklarıyla yerinde değiştirilir.
      - *_std kolonları clean kopyaya eklenmez.
    """
    df_clean = df_original.copy()

    for col in cols_multi + cols_single:
        std_col = col + "_std"
        if (col in df_clean.columns) and (std_col in df_std.columns):
            df_clean[col] = df_std[std_col]

    # Clean kopyada *_std kolonları olmasın
    std_cols = [c for c in df_clean.columns if c.endswith("_std")]
    if std_cols:
        df_clean = df_clean.drop(columns=std_cols)

    return df_clean

# --------- Çalıştırma ---------
if __name__ == "__main__":
    df = pd.read_excel(DATA_PATH, sheet_name=SHEET_NAME)

    # 1) Orijinal + *_std sütunlarını üret
    df_std = run_standardization(
        df,
        cols_multi=COLS_MULTI,
        cols_single=COLS_SINGLE,
        min_count_default=MIN_COUNT_DEFAULT,
        manual_aliases=MANUAL_ALIASES
    )

    # 2) Temiz kopya: kolon isimleri ve POZİSYONLARI aynen korunur; içerikler std (ASCII) ile değiştirilir
    df_clean = make_clean_copy_keep_order(df_original=df, df_std=df_std,
                                          cols_multi=COLS_MULTI, cols_single=COLS_SINGLE)

    # ---- Kısa kontrol ----
    print("\n[Kontrol] df kolon sırası korunuyor mu?")
    print("Aynı mı:", list(df.columns) == list(df_clean.columns))
    print("Standardize edilen kolonlar:", COLS_MULTI + COLS_SINGLE)
    print("ASCII çıktı aktif mi?:", OUTPUT_ASCII)

    # 3) Kaydet
    with pd.ExcelWriter(OUT_STD_ALL, engine="openpyxl") as writer:
        df_std.to_excel(writer, index=False)
    with pd.ExcelWriter(OUT_CLEAN, engine="openpyxl") as writer:
        df_clean.to_excel(writer, index=False)

    print(f"\n✅ Kaydedildi:")
    print(f" - {OUT_STD_ALL}  (orijinal + *_std)")
    print(f" - {OUT_CLEAN}    (clean: kolon sırası AYNI, içerik standardize, ASCII)")



[Kontrol] df kolon sırası korunuyor mu?
Aynı mı: True
Standardize edilen kolonlar: ['kronik_hastalik', 'alerji', 'tanilar', 'uygulama_yerleri']
ASCII çıktı aktif mi?: True

✅ Kaydedildi:
 - Clean_Data_Case_DT_2025_std.xlsx  (orijinal + *_std)
 - Clean_Data_Case_DT_2025_clean.xlsx    (clean: kolon sırası AYNI, içerik standardize, ASCII)


In [5]:
# === Clean & Deduplicate Pipeline (ENTEGRE + demografik doldurma + taraf uyumu + GRUP ANAHTARI) ===
# Girdi  -> /kaggle/working/Clean_Data_Case_DT_2025_clean.xlsx
# Çıktı  -> /kaggle/working/Clean_Data_Case_DT_2025_clean_final.(xlsx|csv)

import re, numpy as np, pandas as pd

IN_PATH  = "/kaggle/working/Clean_Data_Case_DT_2025_clean.xlsx"
OUT_XLSX = "/kaggle/working/Clean_Data_Case_DT_2025_clean_final.xlsx"
OUT_CSV  = "/kaggle/working/Clean_Data_Case_DT_2025_clean_final.csv"

# (Opsiyonel) 5 dakikalık süre anomali filtresi bayrağı
APPLY_DROP_5 = True

# ---------------- Helpers ----------------
def to_int_safe(x):
    if pd.isna(x): return np.nan
    m = re.search(r"(\d+)", str(x))
    return int(m.group(1)) if m else np.nan

def tidy_trailing_punct(s):
    if pd.isna(s): return s
    t = str(s).strip()
    t = re.sub(r"[,\s]+$", "", t)
    t = re.sub(r"\s+", " ", t)
    return t.strip()

def split_tokens(s):
    if pd.isna(s) or str(s).strip()=="":
        return []
    return [p.strip() for p in re.split(r"[;,]", str(s)) if p and p.strip()]

def join_tokens(tokens):
    toks = sorted(set([t for t in tokens if t]))
    return ", ".join(toks) if toks else np.nan

def mode_or_first(series: pd.Series):
    vc = series.dropna().astype(str).value_counts()
    if vc.empty: return np.nan
    top = vc.max()
    cands = sorted(vc[vc==top].index.tolist())
    return cands[0]

def collapse_hyphen_repeat(s):
    if pd.isna(s): return s
    t = str(s)
    if "-" in t:
        parts = [p.strip() for p in t.split("-") if p.strip()]
        if len(parts)==2 and parts[0].lower()==parts[1].lower():
            return parts[0]
    return t

# ---------------- 1) Load ----------------
df = pd.read_excel(IN_PATH)
rows_in = len(df)
cols_order = df.columns.tolist()

# snake_case isimler (varsa dönüştür)
rename_map = {
    'HastaNo':'hasta_no','Yas':'yas','Cinsiyet':'cinsiyet','KanGrubu':'kan_grubu','Uyruk':'uyruk',
    'KronikHastalik':'kronik_hastalik','Bolum':'bolum','Alerji':'alerji','Tanilar':'tanilar',
    'TedaviAdi':'tedavi_adi','TedaviSuresi':'tedavi_suresi',
    'UygulamaYerleri':'uygulama_yerleri','UygulamaSuresi':'uygulama_suresi'
}
df.rename(columns={k:v for k,v in rename_map.items() if k in df.columns}, inplace=True)

# ---------------- 2) Aynı hasta içinde boşları doldur (demografik + kronik) ----------------
def fill_by_patient_mode(df, patient_col="hasta_no",
                         cols_to_fill=("kronik_hastalik","cinsiyet","kan_grubu","uyruk","yas")):
    df = df.copy()
    if patient_col not in df.columns:
        return df
    g = df.groupby(patient_col, dropna=False)
    for col in cols_to_fill:
        if col not in df.columns: 
            continue
        mode_map = g[col].transform(mode_or_first)
        mask_na = df[col].isna() | (df[col].astype(str).str.strip()=="")
        df.loc[mask_na, col] = mode_map[mask_na]
    return df

df = fill_by_patient_mode(df)

# ---------------- 3) Kozmetik temizlik ----------------
if "tedavi_adi" in df.columns:
    df["tedavi_adi"] = df["tedavi_adi"].apply(tidy_trailing_punct).apply(collapse_hyphen_repeat)

msk_pat = r"(men[iı]sk[üu]s|diz|kal(c|ç)a|omuz|aş[iı]l|ayak.*bile|bilek|dirsek|el|sirt|boyun|trokanter)"
if "bolum" in df.columns:
    tanilar_l = df.get("tanilar", pd.Series([""]*len(df))).fillna("").str.lower()
    tedavi_l  = df.get("tedavi_adi", pd.Series([""]*len(df))).fillna("").str.lower()
    is_msk = tanilar_l.str.contains(msk_pat, regex=True) | tedavi_l.str.contains(msk_pat, regex=True)
    def drop_solunum_in_msk(val, is_msk_flag):
        if not is_msk_flag or pd.isna(val): 
            return val
        toks = split_tokens(val)
        toks = [t for t in toks if t.strip().lower() != "solunum merkezi"]
        return join_tokens(toks) if toks else val
    df["bolum"] = [drop_solunum_in_msk(v, flag) for v, flag in zip(df["bolum"], is_msk)]

if "uygulama_yerleri" in df.columns:
    df["uygulama_yerleri"] = df["uygulama_yerleri"].apply(tidy_trailing_punct)

# ---------------- 3.45) tedavi_adi'ndaki taraf bilgisini uygulama_yerleri ile hizala ----------------
SIDE_SAG = re.compile(r"\b(sağ|sag)\b", re.I)
SIDE_SOL = re.compile(r"\bsol\b", re.I)
ANAT_PAT = re.compile(r"\b(diz|kal(c|ç)a|ayak(?:\s*|.*)bilek|bilek|omuz|dirsek|el|ayak|bacak|uyluk)\b", re.I)

def detect_side(text):
    if not isinstance(text, str): 
        return None
    t = text.lower()
    if SIDE_SOL.search(t): return "sol"
    if SIDE_SAG.search(t): return "sag"
    return None

def enforce_side_on_site(site, target_side):
    if pd.isna(site) or not target_side:
        return site
    s = str(site)
    s_low = s.lower()
    if "bilateral" in s_low:
        return s
    if target_side == "sol":
        s2 = re.sub(r"\b(sağ|sag)\b", "sol", s, flags=re.I)
    else:  # "sag"
        s2 = re.sub(r"\bsol\b", "sag", s, flags=re.I)
    s2_low = s2.lower()
    if not (SIDE_SOL.search(s2_low) or SIDE_SAG.search(s2_low)):
        if ANAT_PAT.search(s2_low):
            s2 = f"{target_side} {s2}"
    s2 = re.sub(r"\s*,\s*", ", ", s2).strip()
    return s2

if {"tedavi_adi","uygulama_yerleri"}.issubset(df.columns):
    target_side_from_tedavi = df["tedavi_adi"].astype(str).apply(detect_side)
    mask_need_side = target_side_from_tedavi.notna()
    df.loc[mask_need_side, "uygulama_yerleri"] = [
        enforce_side_on_site(site, side) 
        for site, side in zip(df.loc[mask_need_side, "uygulama_yerleri"], 
                              target_side_from_tedavi[mask_need_side])
    ]

# ---------------- 3.4) (YENİ) GRUP ANAHTARI normalizasyonu ----------------
TURK_MAP = str.maketrans({
    "ç":"c","Ç":"c","ğ":"g","Ğ":"g","ı":"i","İ":"i","I":"i",
    "ö":"o","Ö":"o","ş":"s","Ş":"s","ü":"u","Ü":"u"
})
EXCLUDE_FOR_GROUP = {"uygulama_yerleri", "uygulama_suresi"}  # gruptan hariç
TEXT_COLS_FOR_KEY = [c for c in df.columns if c not in EXCLUDE_FOR_GROUP]

def norm_key(v):
    if pd.isna(v): return ""
    t = str(v).lower().translate(TURK_MAP)
    t = re.sub(r"\s+", " ", t).strip()
    t = re.sub(r"[\W_]+", "", t)
    return t

for c in TEXT_COLS_FOR_KEY:
    df[f"_gk_{c}"] = df[c].apply(norm_key)

GK_COLS = [f"_gk_{c}" for c in TEXT_COLS_FOR_KEY]

# ---------------- 3.5) Hasta+Tedavi bazında uygulama_yerleri’ni tutarlılaştır ----------------
WHOLE_BODY = re.compile(r"\bt[üu]m v[üu]cut\b", flags=re.I)
SPECIFIC   = re.compile(r"\b(diz|ayak.*bile|bilek|kal(c|ç)a|omuz|bel|boyun|dirsek|el|s[ıi]rt|trapez|skapula)\b", re.I)

def classify_site(text):
    if text is None or str(text).strip()=="":
        return 0
    t = str(text).strip().lower()
    if WHOLE_BODY.search(t): return 1
    if SPECIFIC.search(t):   return 2
    return 1

def split_sites(s):
    if pd.isna(s) or str(s).strip()=="":
        return []
    return sorted({p.strip() for p in re.split(r"[;,]", str(s)) if p and p.strip()})

def choose_representative_site(series):
    s = series.fillna("").astype(str)
    if s.empty: return np.nan
    cls = s.apply(classify_site)
    max_cls = cls.max()
    cand = s[cls == max_cls]
    if max_cls == 2:
        records = [(len(split_sites(v)), v) for v in cand]
        max_size = max(sz for sz,_ in records)
        cand2 = [v for sz,v in records if sz == max_size]
        vc = pd.Series(cand2).value_counts()
        top = vc.max()
        cands = sorted(vc[vc==top].index.tolist())
        return cands[0]
    else:
        vc = cand.value_counts()
        top = vc.max()
        cands = sorted(vc[vc==top].index.tolist())
        return cands[0] if cands else np.nan

group_keys_ht = [c for c in ["hasta_no","tedavi_adi"] if c in df.columns]
if group_keys_ht:
    rep_site = df.groupby(group_keys_ht)["uygulama_yerleri"].transform(choose_representative_site)
    is_whole = df["uygulama_yerleri"].fillna("").str.lower().str.contains(WHOLE_BODY)
    is_empty = df["uygulama_yerleri"].isna() | (df["uygulama_yerleri"].astype(str).str.strip().eq(""))
    need_fill = is_whole | is_empty
    df.loc[need_fill, "uygulama_yerleri"] = rep_site[need_fill]

# ---------------- 4) 'Tüm Vücut' saçmalığını at (spesifik varsa) [GK_COLS ile grup] ----------------
def drop_whole_body_if_specific_exists(df):
    df = df.copy()
    if "uygulama_yerleri" not in df.columns: 
        return df, 0
    specific_pat = SPECIFIC.pattern
    whole_body_pat = WHOLE_BODY.pattern
    grp = df.groupby(GK_COLS, dropna=False)
    has_specific = grp["uygulama_yerleri"].transform(
        lambda s: s.fillna("").str.lower().str.contains(specific_pat, regex=True).any()
    )
    is_whole = df["uygulama_yerleri"].fillna("").str.lower().str.contains(whole_body_pat, regex=True)
    drop_idx = df.index[has_specific & is_whole]
    return df.drop(index=drop_idx), len(drop_idx)

df, dropped_whole = drop_whole_body_if_specific_exists(df)

# ---------------- 5) Süre 5 anomalisini at (grupta >5 varsa) [GK_COLS ile grup] ----------------
def drop_duration_5_anomaly(df):
    df = df.copy()
    if "uygulama_suresi" not in df.columns:
        return df, 0
    df["_dur"] = df["uygulama_suresi"].apply(to_int_safe)
    grp = df.groupby(GK_COLS, dropna=False)
    has_gt5 = grp["_dur"].transform(lambda s: (s > 5).any())
    mask_drop5 = has_gt5 & (df["_dur"] == 5)
    out = df.loc[~mask_drop5].copy()
    out.drop(columns=["_dur"], inplace=True, errors="ignore")
    return out, int(mask_drop5.sum())

if APPLY_DROP_5:
    df, dropped_5 = drop_duration_5_anomaly(df)
else:
    dropped_5 = 0

# ---------------- 6) Tekilleştirme (grup başına tek satır) [GK_COLS ile grup] ----------------
# >>> GÜNCELLENEN, DAHA GÜVENLİ SÜRÜM ( _dur_diff OLUŞTURMADAN ) <<<
def collapse_groups_keep_superset_and_mode(df):
    """
    GK_COLS aynı olan kayıtları tekilleştirir.
    - Uygulama yeri: en kapsamlı (token sayısı en çok) tercih edilir.
    - Süre: grubun modu (eşitlikte maksimum) 'temsilî süre' olarak alınır.
    - En kapsamlı adaylarda temsilî süre yoksa:
        * en kapsamlılar içinden rep_dur'a |dur-rep_dur| EN YAKIN olanı seç,
        * çıktıdaki 'uygulama_suresi'ni rep_dur'a normalize et.
    """
    def _split_sites(s):
        if pd.isna(s) or str(s).strip()=="":
            return []
        return sorted({p.strip() for p in re.split(r"[;,]", str(s)) if p and p.strip()})

    def pick_rep_duration(s: pd.Series) -> float:
        vc = s.value_counts(dropna=True)
        if vc.empty: return np.nan
        top = vc.max()
        cands = vc[vc==top].index.tolist()
        return float(np.nanmax(cands))  # eşitlikte maksimum

    df = df.copy()
    df["_dur"] = df["uygulama_suresi"].apply(to_int_safe).astype("float") if "uygulama_suresi" in df.columns else np.nan
    df["_sites"] = df.get("uygulama_yerleri", pd.Series([np.nan]*len(df))).apply(_split_sites)
    df["_sites_size"] = df["_sites"].apply(len).astype(int)

    rows = []
    for _, gdf in df.groupby(GK_COLS, dropna=False):
        gdf = gdf.copy()
        rep_dur = pick_rep_duration(gdf["_dur"]) if "_dur" in gdf else np.nan

        max_site_sz = gdf["_sites_size"].max()
        g_best = gdf[gdf["_sites_size"] == max_site_sz]

        # 1) En kapsamlı + rep_dur eşleşmesi varsa, onu seç
        cand = g_best[g_best["_dur"] == rep_dur]
        if len(cand) == 0:
            # 2) En kapsamlılar içinde rep_dur'a EN YAKIN süreyi seç (kolon eklemeden)
            if not np.isnan(rep_dur):
                diffs = (g_best["_dur"] - rep_dur).abs()
                # Eşitlikte küçük süreyi tercih etmek için stable sort (mergesort) + ikincil anahtar
                order = np.lexsort((g_best["_dur"].values, diffs.values))
                idx = g_best.index[order][0]
                cand = g_best.loc[[idx]]
            else:
                cand = g_best.head(1)

        if len(cand) == 0:
            cand = gdf[gdf["_dur"] == rep_dur]
            if len(cand) == 0:
                cand = gdf.head(1)

        row = cand.iloc[0].copy()
        if not np.isnan(rep_dur):
            row["uygulama_suresi"] = int(rep_dur)

        rows.append(row)

    out = pd.DataFrame(rows).copy()
    # Yardımcı kolonları (ve varsa yanlışlıkla sızan _dur_diff'i) temizle
    out.drop(columns=["_dur","_sites","_sites_size","_dur_diff"], inplace=True, errors="ignore")
    return out

df_final = collapse_groups_keep_superset_and_mode(df)

# ---------------- 7) Save ----------------
rows_out = len(df_final)
print({
    "rows_in": rows_in,
    "dropped_whole_body": dropped_whole,
    "dropped_duration_5": dropped_5,
    "rows_out": rows_out
})

# Kolon sırasını koru ve _gk_* yardımcılarını at
for c in df_final.columns:
    if c not in cols_order:
        cols_order.append(c)
df_final = df_final[[c for c in cols_order if c in df_final.columns]]
df_final.drop(columns=[c for c in df_final.columns if c.startswith("_gk_")],
              inplace=True, errors="ignore")

df_final.to_excel(OUT_XLSX, index=False)
df_final.to_csv(OUT_CSV, index=False)
print("✅ Kaydedildi:")
print(" -", OUT_XLSX)
print(" -", OUT_CSV)


{'rows_in': 2235, 'dropped_whole_body': 4, 'dropped_duration_5': 351, 'rows_out': 511}
✅ Kaydedildi:
 - /kaggle/working/Clean_Data_Case_DT_2025_clean_final.xlsx
 - /kaggle/working/Clean_Data_Case_DT_2025_clean_final.csv


In [6]:
# ============================
# Cinsiyet Doldurma + Boşluk Temizleme (Kaggle)
# ============================
import re
import numpy as np
import pandas as pd

# ---- YOLLAR ----
# Excel/CSV her ikisini de destekleyelim; varsa Excel'i, yoksa CSV'yi oku.
PREFERRED = [
    "/kaggle/working/Clean_Data_Case_DT_2025_clean_final.xlsx",
    "/kaggle/working/Clean_Data_Case_DT_2025_clean_final.csv",
]

def _read_any(preferred_paths):
    for p in preferred_paths:
        try:
            if p.lower().endswith(".xlsx"):
                return pd.read_excel(p), p
            elif p.lower().endswith(".csv"):
                return pd.read_csv(p), p
        except Exception:
            pass
    raise FileNotFoundError("Okunabilir bir dosya bulunamadı. Yolu güncelleyin.")

df, in_path = _read_any(PREFERRED)
print("OKUNAN:", in_path)
print("Şekil:", df.shape)

# ---- Yardımcılar ----
def _is_na(x):
    if x is None:
        return True
    s = str(x).strip()
    if s == "" or s.lower() in {"nan", "none", "null"}:
        return True
    return pd.isna(x)

def _norm(s: str) -> str:
    """Türkçe karakterleri koruyarak sadece küçük harfe çeker ve boşlukları sıkıştırır."""
    if _is_na(s): 
        return ""
    s = str(s).casefold()
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ---- Kolon isimleri (dosyan göre uyumlu) ----
COL_CINS = "cinsiyet"
COL_KRON = "kronik_hastalik"
COL_TANI = "tanilar"
COL_ALER = "alerji"

missing_cols = [c for c in [COL_CINS, COL_KRON, COL_TANI, COL_ALER] if c not in df.columns]
if missing_cols:
    raise ValueError(f"Beklenen kolon(lar) yok: {missing_cols}")

# ===========================================
# 1) Alerji & Kronik Hastalık NaN -> "yok"
# ===========================================
for col in [COL_ALER, COL_KRON]:
    df[col] = df[col].apply(lambda x: "yok" if _is_na(x) else str(x).strip())

# ===========================================
# 2) Cinsiyet: Kural Tabanlı + Oransal Doldurma
# ===========================================
# --- Kural kümeleri (genişletilebilir) ---
# Kadın için daha ayırt edici ipuçları:
female_patterns = [
    r"\b(rahim|uterus|serviks|servical|servicalis)\b",
    r"\b(over|ovary|ovaryan|ovarian|over kisti|polikistik over|pko)\b",
    r"\b(meme|endometrioz|endometriozis)\b",
    r"\b(gebelik|hamile|gravid|lohusalık|menopoz|adet|dismenore)\b",
    r"\b(tiroid|hipotiroid|hipertiroid|tiroidit)\b",
    r"\b(myasteni|myasthenia)\b",
    r"\b(polimiyozit|polimiyozit)\b",
]

# Erkek için daha ayırt edici ipuçları:
male_patterns = [
    r"\b(prostat|bph|benign prostat hiperplazisi)\b",
    r"\b(testis|testik|penil|epididim|varikosel|erkek infertil)\b",
    r"\b(hipospad|hypospad)\b",
    r"\b(duchenne|becker)\b",  # musküler distrofi alt tipleri
]

female_re = re.compile("|".join(female_patterns))
male_re   = re.compile("|".join(male_patterns))

# --- Cinsiyeti normalize et (sadece 'Kadın'/'Erkek' kalsın) ---
def _std_gender(x):
    if _is_na(x): 
        return np.nan
    s = str(x).strip().casefold()
    if s in {"kadin", "kadın", "female", "f", "k", "woman"}:
        return "Kadın"
    if s in {"erkek", "male", "m", "e", "man"}:
        return "Erkek"
    return np.nan

df[COL_CINS] = df[COL_CINS].apply(_std_gender)

# --- Kural tabanlı infer: sadece boş cinsiyetler için uygula ---
mask_missing = df[COL_CINS].isna()

# Tanı + kronik birleştirip normalize et
comb_text = (
    df[COL_KRON].fillna("").astype(str) + " " + df[COL_TANI].fillna("").astype(str)
).apply(_norm)

def infer_gender_from_text(txt: str):
    if not txt:
        return np.nan
    has_f = bool(female_re.search(txt))
    has_m = bool(male_re.search(txt))
    if has_f and not has_m:
        return "Kadın"
    if has_m and not has_f:
        return "Erkek"
    # Çakışma veya ipucu yoksa boş bırak (sonra oransal doldurulacak)
    return np.nan

df.loc[mask_missing, COL_CINS] = comb_text[mask_missing].apply(infer_gender_from_text)

# --- Geriye kalan boşları oransal doldur ---
mask_missing = df[COL_CINS].isna()
if mask_missing.any():
    vc = df[COL_CINS].value_counts(dropna=True)
    n_known = vc.sum()
    if n_known == 0:
        p_female = 0.5
        p_male   = 0.5
    else:
        p_female = float(vc.get("Kadın", 0)) / n_known
        p_male   = float(vc.get("Erkek", 0)) / n_known
        # güvenlik: oranlardan biri 0 ise çok küçük bir epsilon ekleyelim
        eps = 1e-9
        p_female = max(p_female, eps)
        p_male   = max(p_male, eps)
        s = p_female + p_male
        p_female, p_male = p_female/s, p_male/s

    rng = np.random.default_rng(42)  # tekrarlanabilirlik
    fill_vals = rng.choice(["Kadın", "Erkek"], size=mask_missing.sum(), p=[p_female, p_male])
    df.loc[mask_missing, COL_CINS] = fill_vals

# ============================
# Kontrol & Kayıt
# ============================
print("\n[Cinsiyet Dağılımı]".upper())
print(df[COL_CINS].value_counts())

# Çıkış dosyaları
out_csv  = "/kaggle/working/dataset_gender_filled.csv"
out_xlsx = "/kaggle/working/dataset_gender_filled.xlsx"
df.to_csv(out_csv, index=False)
try:
    df.to_excel(out_xlsx, index=False)
except Exception as e:
    print("Excel'e yazarken uyarı:", e)

print("\nKaydedildi:")
print(out_csv)
print(out_xlsx)


OKUNAN: /kaggle/working/Clean_Data_Case_DT_2025_clean_final.xlsx
Şekil: (511, 13)

[CINSIYET DAĞILIMI]
cinsiyet
Kadın    304
Erkek    207
Name: count, dtype: int64

Kaydedildi:
/kaggle/working/dataset_gender_filled.csv
/kaggle/working/dataset_gender_filled.xlsx


In [7]:
# ============================================
# Güvenli Doldurma Pipeline (Hasta -> Grup -> Bilinmiyor)
# ============================================
import numpy as np
import pandas as pd

# ---- Girdi yolları (varsa hangisi bulunursa onu okuyacak) ----
PREFERRED = [
    "/kaggle/working/Clean_Data_Case_DT_2025_clean_final.xlsx"
]

def _read_any(paths):
    for p in paths:
        try:
            if p.lower().endswith(".xlsx"):
                return pd.read_excel(p), p
            elif p.lower().endswith(".csv"):
                return pd.read_csv(p), p
        except Exception:
            pass
    raise FileNotFoundError("Okunabilir bir dosya bulunamadı; PREFERRED listesini güncelleyin.")

df, in_path = _read_any(PREFERRED)
print("OKUNAN:", in_path)
print("Şekil:", df.shape)

# ---- Beklenen kolonlar ----
REQUIRED = [
    "hasta_no","yas","cinsiyet","kan_grubu","uyruk","kronik_hastalik",
    "bolum","alerji","tanilar","tedavi_adi","tedavi_suresi",
    "uygulama_yerleri","uygulama_suresi"
]
missing_cols = [c for c in REQUIRED if c not in df.columns]
if missing_cols:
    raise ValueError(f"Eksik kolon(lar): {missing_cols}")

# ---- Yardımcılar ----
def is_na_like(x):
    if x is None:
        return True
    s = str(x).strip()
    if s == "" or s.lower() in {"nan","none","null"}:
        return True
    return pd.isna(x)

def safe_mode(series: pd.Series):
    """En sık değeri döndürür (NaN hariç). Boşsa None."""
    s = series.dropna()
    if s.empty:
        return None
    return s.mode(dropna=True).iloc[0] if not s.mode(dropna=True).empty else None

def unique_if_consistent(series: pd.Series):
    """Seride NaN hariç tek bir değer varsa onu döndür; yoksa None."""
    s = series.dropna().unique()
    return s[0] if len(s) == 1 else None

def fill_from_patient_consensus(df, target_col, id_col="hasta_no"):
    """
    Aynı hasta_no'da target_col için çelişki yoksa (tek değer) eksikleri o değerle doldurur.
    """
    before_missing = df[target_col].isna().sum()
    # hasta_no -> değer eşlemesi (tutarlıysa)
    cons_map = (
        df.groupby(id_col)[target_col]
          .apply(unique_if_consistent)
          .dropna()
          .to_dict()
    )
    mask = df[target_col].isna() & df[id_col].isin(cons_map.keys())
    df.loc[mask, target_col] = df.loc[mask, id_col].map(cons_map)
    filled = before_missing - df[target_col].isna().sum()
    return int(filled)

def fill_from_group_mode(df, target_col, group_cols, min_count=3, min_ratio=0.7, unanimous=False):
    """
    Grup bazlı doldurma.
    - mode değer sayısı >= min_count olmalı.
    - mode oranı >= min_ratio (ör. 0.7) olmalı.
    - unanimous=True ise gruptaki non-null değerlerin tamamı tek değerse doldur.
    """
    before_missing = df[target_col].isna().sum()

    grp = df.groupby(group_cols)[target_col]
    stats = grp.agg(
        non_null_count=lambda s: s.notna().sum(),
        nunique=lambda s: s.dropna().nunique(),
        mode_val=lambda s: safe_mode(s),
    ).reset_index()

    if unanimous:
        # Tam mutabakat: non-null > 0 ve nunique == 1
        ok = stats.query("non_null_count >= @min_count and nunique == 1").copy()
        ok["use_val"] = ok["mode_val"]
    else:
        # Oran kontrolü: mode / non_null_count >= min_ratio
        def _mode_ratio(sub):
            s = sub.dropna()
            if len(s) == 0:
                return 0.0
            m = safe_mode(s)
            return (s == m).mean() if m is not None else 0.0

        stats["mode_ratio"] = grp.apply(lambda s: _mode_ratio(s)).values
        ok = stats.query("non_null_count >= @min_count and mode_ratio >= @min_ratio").copy()
        ok["use_val"] = ok["mode_val"]

    # Hızlı join için anahtarı üret
    key_df = df[group_cols].copy()
    key_df["_row_id"] = np.arange(len(df))
    ok_key = ok[group_cols + ["use_val"]].copy()

    # Sadece eksik satırlar
    to_fill = df[target_col].isna()
    merged = key_df.loc[to_fill].merge(ok_key, on=group_cols, how="left")
    # Eşleşenlerde doldur
    hit = merged["use_val"].notna()
    df.loc[merged.loc[hit, "_row_id"], target_col] = merged.loc[hit, "use_val"].values

    filled = before_missing - df[target_col].isna().sum()
    return int(filled)

# ---- Başlangıç istatistikleri ----
def null_counts(d):
    return d.isna().sum()[["kan_grubu","bolum","tanilar","uygulama_yerleri"]]

print("\nBAŞLANGIÇ NaN SAYILARI:")
print(null_counts(df))

# ============================================
# 1) Hasta bazlı doldurma (en güvenilir)
# ============================================
filled_patient = {"bolum":0, "tanilar":0, "uygulama_yerleri":0}
for col in ["bolum", "tanilar", "uygulama_yerleri"]:
    filled_patient[col] = fill_from_patient_consensus(df, col, id_col="hasta_no")

print("\n[Hasta bazlı doldurulan]  ", filled_patient)

# ============================================
# 2) Grup bazlı doldurma (güvenlik eşikleriyle)
# ============================================
filled_group = {"bolum":0, "tanilar":0, "uygulama_yerleri":0}

# 2.a) BÖLÜM: tedavi_adi + tedavi_suresi -> mode, min_count>=3, ratio>=0.7
filled_group["bolum"] = fill_from_group_mode(
    df, "bolum",
    group_cols=["tedavi_adi","tedavi_suresi"],
    min_count=3, min_ratio=0.7, unanimous=False
)

# 2.b) UYGULAMA_YERLERİ: tedavi_adi + tedavi_suresi -> mode, min_count>=3, ratio>=0.7
filled_group["uygulama_yerleri"] = fill_from_group_mode(
    df, "uygulama_yerleri",
    group_cols=["tedavi_adi","tedavi_suresi"],
    min_count=3, min_ratio=0.7, unanimous=False
)

# 2.c) TANI: Çok dikkatli — sadece TAM MUTABAKAT ile doldur.
#          (tedavi_adi + tedavi_suresi + uygulama_yerleri) içinde nunique==1 ve count>=5
filled_group["tanilar"] = fill_from_group_mode(
    df, "tanilar",
    group_cols=["tedavi_adi","tedavi_suresi","uygulama_yerleri"],
    min_count=5, min_ratio=1.0, unanimous=True
)

print("\n[Grup bazlı doldurulan]   ", filled_group)

# ============================================
# 3) Kalan eksikler için güvenli yedek: 'bilinmiyor'
# ============================================
# Kan grubu için tahmin yok -> direkt 'bilinmiyor'
df["kan_grubu"] = df["kan_grubu"].apply(lambda x: "bilinmiyor" if is_na_like(x) else x)

# Diğer hedef kolonlarda kalan NaN -> 'bilinmiyor'
for col in ["bolum","tanilar","uygulama_yerleri"]:
    df[col] = df[col].apply(lambda x: "bilinmiyor" if is_na_like(x) else x)

print("\nSON NaN SAYILARI (hepsi 0 olmalı):")
print(df[["kan_grubu","bolum","tanilar","uygulama_yerleri"]].isna().sum())

# ============================================
# Raporlama
# ============================================
print("\n=== ÖZET ===")
print("Hasta bazlı doldurulanlar:", filled_patient)
print("Grup bazlı doldurulanlar :", filled_group)

# Örnek dağılımlar (bakış)
for col in ["bolum","uygulama_yerleri"]:
    vc = df[col].value_counts().head(10)
    print(f"\nTop {col} değerleri:")
    print(vc)

# ============================================
# Kayıt
# ============================================
out_csv  = "/kaggle/working/dataset_imputed.csv"
out_xlsx = "/kaggle/working/dataset_imputed.xlsx"
df.to_csv(out_csv, index=False)
try:
    df.to_excel(out_xlsx, index=False)
except Exception as e:
    print("Excel yazım uyarısı:", e)

print("\nKaydedildi:")
print(out_csv)
print(out_xlsx)


OKUNAN: /kaggle/working/Clean_Data_Case_DT_2025_clean_final.xlsx
Şekil: (511, 13)

BAŞLANGIÇ NaN SAYILARI:
kan_grubu           136
bolum                 2
tanilar              20
uygulama_yerleri     34
dtype: int64

[Hasta bazlı doldurulan]   {'bolum': 0, 'tanilar': 4, 'uygulama_yerleri': 11}

[Grup bazlı doldurulan]    {'bolum': 0, 'tanilar': 0, 'uygulama_yerleri': 0}

SON NaN SAYILARI (hepsi 0 olmalı):
kan_grubu           0
bolum               0
tanilar             0
uygulama_yerleri    0
dtype: int64

=== ÖZET ===
Hasta bazlı doldurulanlar: {'bolum': 0, 'tanilar': 4, 'uygulama_yerleri': 11}
Grup bazlı doldurulanlar : {'bolum': 0, 'tanilar': 0, 'uygulama_yerleri': 0}

Top bolum değerleri:
bolum
Fiziksel Tıp Ve Rehabilitasyon                    313
Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi    128
Ortopedi Ve Travmatoloji                           40
İç Hastalıkları                                     9
Nöroloji                                            5
Kardiyoloji            

In [8]:
# ============================================================
# Tek Pipeline: Cinsiyet Doldurma + Güvenli İmputasyon
# (+ Alerji/Kronik/Tanı Normalizasyonları)
# (Hasta -> Grup -> 'bilinmiyor')
# ============================================================
# -*- coding: utf-8 -*-
import re
import numpy as np
import pandas as pd

# ---- Girdi yolları (ilk bulunanı okur) ----
PREFERRED = [
    "/kaggle/working/Clean_Data_Case_DT_2025_clean_final.xlsx",
]

def _read_any(paths):
    for p in paths:
        try:
            if p.lower().endswith(".xlsx"):
                return pd.read_excel(p), p
            elif p.lower().endswith(".csv"):
                return pd.read_csv(p), p
        except Exception:
            pass
    raise FileNotFoundError("Okunabilir bir dosya bulunamadı; PREFERRED listesini güncelleyin.")

df, in_path = _read_any(PREFERRED)
print("OKUNAN:", in_path)
print("Şekil:", df.shape)

# ---- Beklenen kolonlar ----
REQUIRED = [
    "hasta_no","yas","cinsiyet","kan_grubu","uyruk","kronik_hastalik",
    "bolum","alerji","tanilar","tedavi_adi","tedavi_suresi",
    "uygulama_yerleri","uygulama_suresi"
]
missing_cols = [c for c in REQUIRED if c not in df.columns]
if missing_cols:
    raise ValueError(f"Eksik kolon(lar): {missing_cols}")

# ============================================================
# 0) Yardımcılar
# ============================================================
def is_na_like(x):
    if x is None:
        return True
    s = str(x).strip()
    if s == "" or s.lower() in {"nan","none","null"}:
        return True
    return pd.isna(x)

def safe_mode(series: pd.Series):
    s = series.dropna()
    if s.empty:
        return None
    m = s.mode(dropna=True)
    return m.iloc[0] if not m.empty else None

def unique_if_consistent(series: pd.Series):
    s = series.dropna().unique()
    return s[0] if len(s) == 1 else None

def _norm(s: str) -> str:
    if is_na_like(s): 
        return ""
    s = str(s).casefold()
    s = re.sub(r"\s+", " ", s).strip()
    return s

# ============================================================
# 1) Alerji & Kronik Hastalık NaN -> "yok"
# ============================================================
for col in ["alerji", "kronik_hastalik"]:
    df[col] = df[col].apply(lambda x: "yok" if is_na_like(x) else str(x).strip())

# ============================================================
# 1.5) NORMALİZASYON EKLEMELERİ (senin istediklerin)
#      - alerji: volteren->voltaren, yer fistigi->yer fıstığı
#      - kronik: hiportiroidizm->hipotiroidizm
#      - tanılar: bel/sırt dönüşümleri (ters yön)
# ============================================================
import unicodedata

def _split_multi_cell(text):
    # Çoklu değerleri parçala (',' veya ';')
    parts = re.split(r"[;,]", str(text))
    # trim + lower (casefold) + boşları at
    return [p.strip().casefold() for p in parts if p and p.strip()]

def _join_multi_cell(items):
    # Tekrarları at, stabil sıralama için set yerine liste filtrele
    seen = set()
    out = []
    for it in items:
        if it not in seen:
            seen.add(it)
            out.append(it)
    return ", ".join(out)

# --- ALERJİ normalizasyonu ---
_alerji_map = {
    "volteren": "voltaren",
    "voltaren": "voltaren",
    "yer fistigi": "yer fıstığı",
    "yer fıstığı": "yer fıstığı",
    "yok": "yok",
}

def normalize_alerji_cell(x):
    if is_na_like(x): 
        return "yok"
    toks = _split_multi_cell(x)
    norm = [_alerji_map.get(tok, tok) for tok in toks]
    return _join_multi_cell(norm)

df["alerji"] = df["alerji"].apply(normalize_alerji_cell)

# --- KRONİK normalizasyonu ---
_kronik_map = {
    "hiportiroidizm": "hipotiroidizm",
    # istersen buraya başka düzeltmeler ekleyebilirsin
}

def normalize_kronik_cell(x):
    if is_na_like(x): 
        return "yok"
    toks = _split_multi_cell(x)
    norm = [_kronik_map.get(tok, tok) for tok in toks]
    return _join_multi_cell(norm)

df["kronik_hastalik"] = df["kronik_hastalik"].apply(normalize_kronik_cell)

# --- TANILAR normalizasyonu (ters yön) ---
def normalize_tani_text_reverse(s: str) -> str:
    if is_na_like(s):
        return ""
    t = str(s).casefold()
    t = re.sub(r"\s+", " ", t).strip()

    # Önce spesifik kalıplar: bel ağrısı → lomber ağrısı
    t = re.sub(r"\bbel ağrısı\b", "lomber ağrısı", t)
    t = re.sub(r"\bbel agrisi\b", "lomber ağrısı", t)  # ASCII varyant

    # Sonra tek kelime karşılıkları (kelime sınırıyla)
    t = re.sub(r"\bbel\b", "lomber", t)

    # Sırt → dorsalji
    t = re.sub(r"\bsırt ağrısı\b", "dorsalji", t)
    t = re.sub(r"\bsirt agrisi\b", "dorsalji", t)      # ASCII varyant
    t = re.sub(r"\bsırt\b", "dorsalji", t)
    t = re.sub(r"\bsirt\b", "dorsalji", t)

    return t

df["tanilar"] = df["tanilar"].astype(str).apply(normalize_tani_text_reverse)

# ============================================================
# 2) Cinsiyet Doldurma (Kural + Oransal)
# ============================================================
COL_CINS, COL_KRON, COL_TANI, COL_ALER = "cinsiyet", "kronik_hastalik", "tanilar", "alerji"

# Kural kümeleri (genişletilebilir)
female_patterns = [
    r"\b(rahim|uterus|serviks|servical|servicalis)\b",
    r"\b(over|ovary|ovaryan|ovarian|over kisti|polikistik over|pko)\b",
    r"\b(meme|endometrioz|endometriozis)\b",
    r"\b(gebelik|hamile|gravid|lohusalık|menopoz|adet|dismenore)\b",
    r"\b(tiroid|hipotiroid|hipertiroid|tiroidit)\b",
    r"\b(myasteni|myasthenia)\b",
    r"\b(polimiyozit|polimiyozit)\b",
]
male_patterns = [
    r"\b(prostat|bph|benign prostat hiperplazisi)\b",
    r"\b(testis|testik|penil|epididim|varikosel|erkek infertil)\b",
    r"\b(hipospad|hypospad)\b",
    r"\b(duchenne|becker)\b",
]
female_re = re.compile("|".join(female_patterns))
male_re   = re.compile("|".join(male_patterns))

def _std_gender(x):
    if is_na_like(x): 
        return np.nan
    s = str(x).strip().casefold()
    if s in {"kadin", "kadın", "female", "f", "k", "woman"}:
        return "Kadın"
    if s in {"erkek", "male", "m", "e", "man"}:
        return "Erkek"
    return np.nan

df[COL_CINS] = df[COL_CINS].apply(_std_gender)

mask_missing = df[COL_CINS].isna()
comb_text = (df[COL_KRON].fillna("").astype(str) + " " + df[COL_TANI].fillna("").astype(str)).apply(_norm)

def infer_gender_from_text(txt: str):
    if not txt:
        return np.nan
    has_f = bool(female_re.search(txt))
    has_m = bool(male_re.search(txt))
    if has_f and not has_m:
        return "Kadın"
    if has_m and not has_f:
        return "Erkek"
    return np.nan

df.loc[mask_missing, COL_CINS] = comb_text[mask_missing].apply(infer_gender_from_text)

mask_missing = df[COL_CINS].isna()
if mask_missing.any():
    vc = df[COL_CINS].value_counts(dropna=True)
    n_known = vc.sum()
    if n_known == 0:
        p_female = p_male = 0.5
    else:
        p_female = float(vc.get("Kadın", 0)) / n_known
        p_male   = float(vc.get("Erkek", 0)) / n_known
        eps = 1e-9
        p_female = max(p_female, eps)
        p_male   = max(p_male, eps)
        s = p_female + p_male
        p_female, p_male = p_female/s, p_male/s
    rng = np.random.default_rng(42)
    fill_vals = rng.choice(["Kadın", "Erkek"], size=mask_missing.sum(), p=[p_female, p_male])
    df.loc[mask_missing, COL_CINS] = fill_vals

print("\n[Cinsiyet Dağılımı]")
print(df[COL_CINS].value_counts())

# ============================================================
# 3) Güvenli Doldurma (Hasta -> Grup -> 'bilinmiyor')
#    Hedef kolonlar: kan_grubu, bolum, tanilar, uygulama_yerleri
# ============================================================
def fill_from_patient_consensus(df, target_col, id_col="hasta_no"):
    before_missing = df[target_col].isna().sum()
    cons_map = (
        df.groupby(id_col)[target_col]
          .apply(unique_if_consistent)
          .dropna()
          .to_dict()
    )
    mask = df[target_col].isna() & df[id_col].isin(cons_map.keys())
    df.loc[mask, target_col] = df.loc[mask, id_col].map(cons_map)
    return int(before_missing - df[target_col].isna().sum())

def fill_from_group_mode(df, target_col, group_cols, min_count=3, min_ratio=0.7, unanimous=False):
    before_missing = df[target_col].isna().sum()
    grp = df.groupby(group_cols)[target_col]
    stats = grp.agg(
        non_null_count=lambda s: s.notna().sum(),
        nunique=lambda s: s.dropna().nunique(),
        mode_val=lambda s: safe_mode(s),
    ).reset_index()

    if unanimous:
        ok = stats.query("non_null_count >= @min_count and nunique == 1").copy()
        ok["use_val"] = ok["mode_val"]
    else:
        def _mode_ratio(sub):
            s = sub.dropna()
            if len(s) == 0: return 0.0
            m = safe_mode(s)
            return (s == m).mean() if m is not None else 0.0
        stats["mode_ratio"] = grp.apply(lambda s: _mode_ratio(s)).values
        ok = stats.query("non_null_count >= @min_count and mode_ratio >= @min_ratio").copy()
        ok["use_val"] = ok["mode_val"]

    key_df = df[group_cols].copy()
    key_df["_row_id"] = np.arange(len(df))
    ok_key = ok[group_cols + ["use_val"]].copy()

    to_fill = df[target_col].isna()
    merged = key_df.loc[to_fill].merge(ok_key, on=group_cols, how="left")
    hit = merged["use_val"].notna()
    df.loc[merged.loc[hit, "_row_id"], target_col] = merged.loc[hit, "use_val"].values

    return int(before_missing - df[target_col].isna().sum())

def null_counts(d):
    return d.isna().sum()[["kan_grubu","bolum","tanilar","uygulama_yerleri"]]

print("\nBAŞLANGIÇ NaN SAYILARI (hedef kolonlar):")
print(null_counts(df))

# 3.1) Hasta bazlı
filled_patient = {"bolum":0, "tanilar":0, "uygulama_yerleri":0}
for col in ["bolum", "tanilar", "uygulama_yerleri"]:
    filled_patient[col] = fill_from_patient_consensus(df, col, id_col="hasta_no")
print("\n[Hasta bazlı doldurulan] ", filled_patient)

# 3.2) Grup bazlı
filled_group = {"bolum":0, "tanilar":0, "uygulama_yerleri":0}
filled_group["bolum"] = fill_from_group_mode(
    df, "bolum",
    group_cols=["tedavi_adi","tedavi_suresi"],
    min_count=3, min_ratio=0.7, unanimous=False
)
filled_group["uygulama_yerleri"] = fill_from_group_mode(
    df, "uygulama_yerleri",
    group_cols=["tedavi_adi","tedavi_suresi"],
    min_count=3, min_ratio=0.7, unanimous=False
)
# Tanılar için sadece TAM MUTABAKAT ve daha sıkı eşik
filled_group["tanilar"] = fill_from_group_mode(
    df, "tanilar",
    group_cols=["tedavi_adi","tedavi_suresi","uygulama_yerleri"],
    min_count=5, min_ratio=1.0, unanimous=True
)
print("\n[Grup bazlı doldurulan]  ", filled_group)

# 3.3) Fallback: 'bilinmiyor'
df["kan_grubu"] = df["kan_grubu"].apply(lambda x: "bilinmiyor" if is_na_like(x) else x)
for col in ["bolum","tanilar","uygulama_yerleri"]:
    df[col] = df[col].apply(lambda x: "bilinmiyor" if is_na_like(x) else x)

print("\nSON NaN SAYILARI (hepsi 0 olmalı):")
print(df[["kan_grubu","bolum","tanilar","uygulama_yerleri"]].isna().sum())

# Kısa dağılım bakışı
for col in ["bolum","uygulama_yerleri"]:
    print(f"\nTop {col} değerleri:")
    print(df[col].value_counts().head(10))

# ============================================================
# Kayıt
# ============================================================
out_csv  = "/kaggle/working/dataset_final_imputed.csv"
out_xlsx = "/kaggle/working/dataset_final_imputed.xlsx"
df.to_csv(out_csv, index=False)
try:
    df.to_excel(out_xlsx, index=False)
except Exception as e:
    print("Excel yazım uyarısı:", e)

print("\nKaydedildi:")
print(out_csv)
print(out_xlsx)


OKUNAN: /kaggle/working/Clean_Data_Case_DT_2025_clean_final.xlsx
Şekil: (511, 13)

[Cinsiyet Dağılımı]
cinsiyet
Kadın    304
Erkek    207
Name: count, dtype: int64

BAŞLANGIÇ NaN SAYILARI (hedef kolonlar):
kan_grubu           136
bolum                 2
tanilar               0
uygulama_yerleri     34
dtype: int64

[Hasta bazlı doldurulan]  {'bolum': 0, 'tanilar': 0, 'uygulama_yerleri': 11}

[Grup bazlı doldurulan]   {'bolum': 0, 'tanilar': 0, 'uygulama_yerleri': 0}

SON NaN SAYILARI (hepsi 0 olmalı):
kan_grubu           0
bolum               0
tanilar             0
uygulama_yerleri    0
dtype: int64

Top bolum değerleri:
bolum
Fiziksel Tıp Ve Rehabilitasyon                    313
Fiziksel Tıp Ve Rehabilitasyon,Solunum Merkezi    128
Ortopedi Ve Travmatoloji                           40
İç Hastalıkları                                     9
Nöroloji                                            5
Kardiyoloji                                         4
Genel Cerrahi                            

In [9]:
# =========================================================
# TOP-K (20/25/30) KARŞILAŞTIR -> EN İYİ K'YI SEÇ -> TEK ÇIKTI
# Kaggle uyumlu (orijinal metin sütunları tamamen kaldırılır)
# =========================================================
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor

# ------------------ Parametreler ------------------
IN_PATH = "/kaggle/working/dataset_final_imputed.xlsx"
TOP_K_LIST = [20, 25, 30]     # denenecek K değerleri
N_SPLITS = 5                  # 5-fold CV
RANDOM_STATE = 42
SCORING = "neg_mean_absolute_error"  # MAE

# ------------------ Yardımcı Fonksiyonlar ------------------
def split_multi(x):
    if pd.isna(x):
        return []
    parts = re.split(r"[;,]", str(x))
    return [p.strip().casefold() for p in parts if p.strip()]

def ohe_multilabel(series):
    mlb = MultiLabelBinarizer()
    lists = series.apply(split_multi)
    arr = mlb.fit_transform(lists)
    cols = [f"{series.name}__{c}" for c in mlb.classes_]
    return pd.DataFrame(arr, index=series.index, columns=cols).astype("int8")

def topk_multilabel(series, top_list):
    lists = series.apply(split_multi)
    top_set = set(top_list)
    data = {}
    for lab in top_list:
        data[f"{series.name}__{lab}"] = lists.apply(lambda L: int(lab in set(L))).astype("int8")
    data[f"{series.name}__other"] = lists.apply(lambda L: int(any(l for l in L if l not in top_set))).astype("int8")
    return pd.DataFrame(data, index=series.index)

def onehot_drop_first(df, col, prefix=None):
    return pd.get_dummies(
        df[col].astype(str).str.strip().str.lower(),
        prefix=(prefix or col),
        drop_first=True,
        dtype="int8"
    )

def build_dataset(df, TOP_K):
    """Verilen TOP_K için tamamen sayısal ve encode edilmiş dataset döndürür."""
    # Sayısallar
    numeric_cols = ["yas", "tedavi_suresi", "uygulama_suresi"]

    # Cinsiyet (binary)
    cinsiyet_enc = (
        df["cinsiyet"].astype(str).str.casefold().str.strip().map({"kadın":1, "erkek":0})
        .astype("Int8").fillna(0)
    ).to_frame("cinsiyet_enc")

    # One-hot (drop_first)
    kan_ohe   = onehot_drop_first(df, "kan_grubu", prefix="kan")
    uyruk_ohe = onehot_drop_first(df, "uyruk",      prefix="uyruk")
    bolum_ohe = onehot_drop_first(df, "bolum",      prefix="bolum")

    # Multi-label tam OHE
    uyg_ohe  = ohe_multilabel(df["uygulama_yerleri"])
    alr_ohe  = ohe_multilabel(df["alerji"])
    kron_ohe = ohe_multilabel(df["kronik_hastalik"])

    # Tanı frekans → Top-K + other
    tani_lists = df["tanilar"].apply(split_multi)
    cnt_tani = Counter(); [cnt_tani.update(lst) for lst in tani_lists]
    tani_top_list = [lab for lab, _ in cnt_tani.most_common(TOP_K)]
    tani_top_ohe = topk_multilabel(df["tanilar"], tani_top_list)

    # Tedavi adı frekans → Top-K + other
    tedavi_clean = df["tedavi_adi"].astype(str).str.strip().str.casefold()
    tedavi_top_list = list(tedavi_clean.value_counts().head(TOP_K).index)
    tedavi_top_or_other = tedavi_clean.where(tedavi_clean.isin(tedavi_top_list), other="other")
    tedavi_ohe = pd.get_dummies(tedavi_top_or_other, prefix="tedavi", drop_first=False, dtype="int8")

    # Tümünü birleştir (yalnızca sayısal + encode edilmişler)
    parts = [
        df[numeric_cols], 
        cinsiyet_enc,
        kan_ohe, uyruk_ohe, bolum_ohe,
        uyg_ohe, alr_ohe, kron_ohe,
        tani_top_ohe, tedavi_ohe
    ]
    Xy = pd.concat(parts, axis=1)
    # Hedef değişkeni ayır
    y = Xy["tedavi_suresi"].astype(float).values
    X = Xy.drop(columns=["tedavi_suresi"])
    # NaN güvenliği (kalırsa): 0 doldur
    X = X.fillna(0)
    return X, y, Xy.columns.tolist()

# ------------------ Veriyi Yükle ------------------
df = pd.read_excel(IN_PATH)
print(f"OKUNAN: {IN_PATH} | Şekil: {df.shape}")

# ------------------ Model & CV Hazırlığı ------------------
cv = KFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)
model = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

results = []   # (TOP_K, mean_MAE, std_MAE, n_features)

# ------------------ Denemeler: 20/25/30 ------------------
for K in TOP_K_LIST:
    X, y, cols = build_dataset(df, K)
    scores = cross_val_score(model, X, y, scoring=SCORING, cv=cv, n_jobs=-1)
    mae_mean = -scores.mean()
    mae_std  =  scores.std()
    results.append((K, mae_mean, mae_std, X.shape[1]))
    print(f"TOP_K={K:>2} | CV MAE={mae_mean:.3f} ± {mae_std:.3f} | n_features={X.shape[1]}")

# ------------------ En iyi TOP_K'yı seç ------------------
best_K, best_mae, best_std, best_feats = sorted(results, key=lambda t: t[1])[0]
print(f"\n>>> EN İYİ TOP_K: {best_K} | MAE={best_mae:.3f} ± {best_std:.3f} | n_features={best_feats}")

# ------------------ En iyi K ile TEK FİNAL veri seti üret ------------------
X_best, y_best, cols_best = build_dataset(df, best_K)
final_df = pd.DataFrame(X_best, columns=[c for c in cols_best if c != "tedavi_suresi"])
final_df["tedavi_suresi"] = y_best  # hedef en sonda

OUT_CSV  = f"/kaggle/working/dataset_model_ready_best_top{best_K}.csv"
OUT_XLSX = f"/kaggle/working/dataset_model_ready_best_top{best_K}.xlsx"
final_df.to_csv(OUT_CSV, index=False)
try:
    final_df.to_excel(OUT_XLSX, index=False)
except Exception as e:
    print("Excel yazım uyarısı:", e)

print("\nKaydedildi (TEK ÇIKTI):")
print(OUT_CSV)
print(OUT_XLSX)


OKUNAN: /kaggle/working/dataset_final_imputed.xlsx | Şekil: (511, 13)
TOP_K=20 | CV MAE=1.770 ± 0.110 | n_features=119
TOP_K=25 | CV MAE=1.731 ± 0.115 | n_features=129
TOP_K=30 | CV MAE=1.745 ± 0.108 | n_features=139

>>> EN İYİ TOP_K: 25 | MAE=1.731 ± 0.115 | n_features=129

Kaydedildi (TEK ÇIKTI):
/kaggle/working/dataset_model_ready_best_top25.csv
/kaggle/working/dataset_model_ready_best_top25.xlsx
